# Quality of Life

> Functions to improve quality of life. Duplicates a subset of those in `EnvDL`'s `core`.

In [ ]:
#| default_exp qol

In [ ]:
#| export


In [ ]:
#| hide
from nbdev.showdoc import *

## Functions for working with files, directories with the intention of caching results.

In [ ]:
#| export
def read_txt(path, 
             **kwargs # Intended to allow for explicit 'encoding' to be passed into open the file
            ):
    if 'encoding' in kwargs.keys():
        print(kwargs)
        with open(path, 'r', encoding  = kwargs['encoding']) as f:
            data = f.read()        
    else:    
        with open(path, 'r') as f:
            data = f.read()
            
    return(data)

In [ ]:
#| export

def print_txt(path):
    print(read_txt(path = path))

In [ ]:
#| export

def read_json(json_path
             ):
    "Read and return json. Used for train/validation/test splits"
    import json
    with open(json_path, 'r') as fp:
        dat = json.load(fp)
    return(dat)

In [ ]:
#| export

def ensure_dir_path_exists(dir_path = '../ext_data' # Directory path to check
                          ):
    "Iteratively check for and create directories to store output. Ideally this would just be os.mkdirs() but that function is not available in this version of python"
    import os
    
    for i in range(2, len(dir_path.split('/'))+1):
        path_part = '/'.join(dir_path.split('/')[0:i])
        if not os.path.exists(path_part):
            os.mkdir(path_part)

In [ ]:
#| export

"Retrieve a previously calculated result. Return None if it cannot be found."
def get_cached_result(
    save_path
):
    import os
    import pickle as pkl
#     import pickle5 as pkl # Using non-base version of pickle 
#                           # conda env with gpu support for tf and torch uses python 3.7.
#                           # Python 3.7 doesn't contain pickle v 5
    if not os.path.exists(save_path):
        cached_result = None
    else:
        with open(save_path, 'rb') as handle:
                cached_result = pkl.load(handle)
    return(cached_result)

In [ ]:
#| export

def put_cached_result(
    save_path,
    save_obj
):
    import pickle as pkl
#     import pickle5 as pkl
#     from EnvDL.core import ensure_dir_path_exists
    ensure_dir_path_exists(dir_path= '/'.join(save_path.split('/')[:-1]) )
    
    with open(save_path, 'wb') as handle:
            pkl.dump(save_obj, 
                     handle, 
                     protocol=4 # version 4 is used instead of 5 because the container
                                # I'm using with tf and torch uses python 3.7 and version
                                # 5 is introduced in 3.8
                    )

In [ ]:
#| export

def remove_matching_files(
    cache_path, # Directory to query
    match_regex_list = ['.*\.pt', 'yhats\.csv', 'loss_df\.csv'], # List of regexes to match (okay if two regexes match the same entry)
    dry_run = True # Print files to be deleted or delete them. 
):
    "Helper function to clear out cache. Remove files from a folder if they match one of a given set of regexes. Ignores directories in directory. Useful for clearing out model artifacts."
    import os
    import re
    # if empty set is provided, match nothing.
    if match_regex_list == []:
        match_regex_list = ['']
    
    files_to_remove = [[e for e in os.listdir(cache_path) if re.match(match_regex, e)
                       ] for match_regex in match_regex_list]
    # make a (potential) list of lists into a flat list
    new_list = []
    for sub_list in files_to_remove:
        new_list = new_list + sub_list
    # ensure it's deduplicated in case two regexes match with the same item
    files_to_remove = list(set(new_list))
    # remove any directories from consideration
    files_to_remove = [e for e in files_to_remove if os.path.isfile(cache_path+e)]
    # sort to make output more pleasant
    files_to_remove.sort()

    if files_to_remove == []:
        print('No files found to remove.')
    else:
        if dry_run:
            print('Command would remove:')
            print('\n'.join(files_to_remove))
        else:
            for file in files_to_remove:
                os.remove(cache_path+file)

# remove_matching_files(
#     cache_path,
#     match_regex_list = ['.*\.pt', 'yhats\.csv', 'loss_df\.csv'],
#     dry_run = False
# )

## Functions for working with DataFrames, especially cleaning them

In [ ]:
#| export

def find_df_shared_cols(df1,# DataFrame 1 
                        df2 # DataFrame 2
                       ):
    shared_cols = [e for e in list(df1) if e in list(df2)]
    return(shared_cols)

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame.from_dict({'a':[0], 'b':['c']})
df2 = pd.DataFrame.from_dict({'a':[1], 'b':[0]})

find_df_shared_cols(df1, df2)

['a', 'b']

In [ ]:
#| export
def find_df_col_mismatches(df1, # DataFrame 1 
                           df2, # DataFrame 2 
                           showtype = True # Whether the data types should be returned
                          ):
    "Identify columns that match in two dataframes but have mismatched data types"
    import numpy as np
    if showtype:
        return [(e, np.dtype(df1[e]), np.dtype(df2[e])) for e in [e for e in list(df2) if e in list(df1)] if (np.dtype(df1[e]) != np.dtype(df2[e])) ]
    else:
        return [e for e in [e for e in list(df2) if e in list(df1)] if (np.dtype(df1[e]) != np.dtype(df2[e])) ]



In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame.from_dict({'a':[0], 'b':['c']})
df2 = pd.DataFrame.from_dict({'a':[1], 'b':[0]})

find_df_col_mismatches(df1, df2)

[('b', dtype('O'), dtype('int64'))]

In [ ]:
#| export
def summarize_col_missing(df):
    "Report the number and percentage of missing values for a DataFrame"
    import pandas as pd
    return(
        pd.DataFrame({'Col'   : [e for e in list(df)],
              'N_miss' : [sum(df[e].isna()) for e in list(df)],
              'Pr_Comp': [round(100*(1-sum(df[e].isna())/len(df[e])), 1) for e in list(df)]})
    )

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.DataFrame.from_dict({'a':[0, 1, 2, 3, 4, np.nan], 
                              'b':[0, 1, 2, 3, 4, 5     ]})

summarize_col_missing(df= df1)

,Col,N_miss,Pr_Comp
0,a,1,83.3
1,b,0,100.0


In [ ]:
#| export

def sanitize_col(df, # DataFrame
                 col, # Column to be sanitized
                 simple_renames= {}, # Replace entries that match the given key with the given value. {'A':'a'}
                 split_renames= {} # Split entries that should be separate rows into two. {'a_b':['a','b']}
                ):
    "Simplify renaming entries in a column for standardizaiton. Particularly useful for irrigation/managment entries which may be intended for humans not computers."
    import pandas as pd
    # simple renames
    for e in simple_renames.keys():
        mask = (df[col] == e)
        df.loc[mask, col] = simple_renames[e]

    # splits
    # pull out the relevant multiname rows, copy, rename, append
    for e in split_renames.keys():
        mask = (df[col] == e)
        temp = df.loc[mask, :] 

        df = df.loc[~mask, :]
        for e2 in split_renames[e]:
            temp2 = temp.copy()
            temp2[col] = e2
            df = df.merge(temp2, how = 'outer')

    return(df)

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame.from_dict({'letters':['a', 'B', 'cd']})

sanitize_col(df = df1, col = 'letters',
                 simple_renames= {'B':'b'},
                 split_renames= {'cd':['c', 'd']}
                )

,letters
0,a
1,b
2,c
3,d


In [ ]:
# utility funciton to update notebook names?
# rename notebooks
# notebook name, cache path, cache dir name
# also needs to search through and update cache path names for all notebooks that depend on it.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()